In [1]:
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 130)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# A complete filelist of the MaxarImagery & Retrolens folders, generated with the unix command:
# find MaxarImagery/ Retrolens/ > ressci201900060-RNC2-Coastal/Nick/ressci201900060-RNC2-Coastal_MaxarImagery_Retrolens_filelist.txt
filelist = pd.read_csv(prefix + "Nick/filelist.txt", header=None).iloc[:,0]
filelist

0                                                                            Gabrielle/
1                                                                      Gabrielle/Orders
2                                                                 Gabrielle/Orders/AOIs
3                               Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4                                    Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                              ...                                      
184918    Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2.aux.xml
184919    Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif.aux.xml
184920    Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_20SEP1980_mosaic.jp2.aux.xml
184921        Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2.ovr
184922                                   Retrolens/Wellington/MasterBlank_withProxy.shx
Name: 0, Length: 184923, dtype: 

In [3]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

df = filelist[filelist.apply(check_filename)].to_frame(name="filename")
df

,filename
29674,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp
29675,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp
29678,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp
29679,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp
29687,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp
...,...
183558,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
183563,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
184667,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
184678,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_Wellington_13FEB2021.shp


In [4]:
images = filelist[filelist.str.contains("/Stack/", case=False) & filelist.str.endswith((".jpg", ".jp2", ".tif"))]
images

31978     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
31979     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
31981       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
31984       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
31998       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
184904            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
184910            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
184913            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
184914            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
184917            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Name: 0, Length: 2484, dtype: ob

In [5]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_"]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    dirname = os.path.dirname(filename)
    RL_dirname = dirname.replace("Stack/", "").replace("Shorelines", "Stack").replace("MaxarImagery/HighFreq", "Retrolens")
    Maxar_dirname = dirname.replace("Imagery/Shorelines", "Imagery/Stack").replace("Shorelines", "Imagery/Stack").replace("Retrolens", "MaxarImagery/HighFreq")
    Maxar_dirname_uppercase = Maxar_dirname.replace("Stack", "STACK")
    Maxar_dirname_outside_Imagery = Maxar_dirname.replace("Imagery/Stack", "Stack")
    all_files_in_folder = images[images.str.startswith((RL_dirname, Maxar_dirname, Maxar_dirname_uppercase, Maxar_dirname_outside_Imagery))]
    if len(all_files_in_folder) == 0:
        return "", 0
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=all_files_in_folder, processor=fuzz_preprocess)
    return match, score

df["matched_image"], df["match_score"] = zip(*df.filename.apply(get_matching_image))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))
df[["filename", "matched_image", "match_score"]].sort_values(by="match_score").to_csv("shoreline_image_matching.csv", index=False)

Perfect matches: 1466
Imperfect matches: 770


In [59]:
df[df.filename.str.startswith("Gabrielle")]

,filename,matched_image,match_score
29674,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp,,0.0
29675,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp,,0.0
29678,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp,,0.0
29679,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp,,0.0
29687,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp,,0.0
...,...,...,...
31683,Gabrielle/Shorelines/Gisborne/Gisborne/Gisborne_16FEB2022.shp,,0.0
31686,Gabrielle/Shorelines/Gisborne/Gisborne/Gisborne_07MAR2023.shp,,0.0
31702,Gabrielle/Shorelines/Gisborne/TolagaBay/TolagaBay_20JAN2023.shp,,0.0
31711,Gabrielle/Shorelines/Gisborne/TolagaBay/TolagaBay_21FEB2023.shp,,0.0


In [63]:
Gabrielle = df[df.filename.str.startswith("Gabrielle")].copy()
Gabrielle_images = filelist[filelist.str.startswith("Gabrielle") & filelist.str.endswith((".jpg", ".jp2", ".tif"))]
len(Gabrielle), len(Gabrielle_images)

(215, 5785)

In [64]:
def get_matching_image(filename):
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
    return match, score

Gabrielle["matched_image"], Gabrielle["match_score"] = zip(*Gabrielle.filename.apply(get_matching_image))
print("Perfect matches:", sum(Gabrielle.match_score == 100))
print("Imperfect matches:", sum(Gabrielle.match_score < 100))

Perfect matches: 144
Imperfect matches: 71


In [65]:
#pd.set_option("display.max_rows",None)
Gabrielle.filename = Gabrielle.filename.str.replace(prefix,"")
Gabrielle.matched_image = Gabrielle.matched_image.str.replace(prefix,"") 
Gabrielle.sort_values(by="match_score")

,filename,matched_image,match_score
30123,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737
31483,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529
30331,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459
30534,Gabrielle/Shorelines/Delivery/PostGabrielle_shorelines_21022023.shp,Gabrielle/Imagery/pre-storm/Bay of Plenty/Tauranga/tauranga-winter-01m-urban-aerial-photos-2022/BD37_500_022023.jpg,60.000000
30053,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462
...,...,...,...
30844,Gabrielle/Shorelines/Auckland/Medlands/Medlands_06FEB2023.shp,Gabrielle/Imagery/pre-storm/Auckland/Medlands/Medlands_06FEB2023.tif,100.000000
30847,Gabrielle/Shorelines/Auckland/Whangapoua/Whangapoua_19FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Whangapoua/Whangapoua_19FEB2023.tif,100.000000
30878,Gabrielle/Shorelines/Hawkes Bay/Mahanga/Mahia_25JAN2023.shp,Gabrielle/Imagery/pre-storm/Hawkes Bay/Mahia/Mahia_25JAN2023.tif,100.000000
30709,Gabrielle/Shorelines/Auckland/LongBay/LongBay_28DEC2022.shp,Gabrielle/Imagery/pre-storm/Auckland/Long Bay/LongBay_28DEC2022.tif,100.000000


In [66]:
index_tiles = filelist[filelist.str.contains("Gabrielle/.+index-tiles.+.shp$")]
index_tiles

4398                      Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
9959                              Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11836                          Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13806                          Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
13925                               Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14066    Gabrielle/Imagery/pre-storm/Waikato/OpitoBay/waikato-03m-rural-aerial-photos-index-tiles-2021-2023/waikato-03m-rural-aerial-ph...
14123    Gabrielle/Imagery/pre-storm/Waikato/WaihiBeach/bay-of-plenty-02m-rural-aerial-photos-index-tiles-2021/bay-of-plenty-02m-rural-...
14368    Gabrielle/Imagery/

In [10]:
Gabrielle_tiles = pd.concat(gpd.read_file(prefix+f) for f in index_tiles)
len(Gabrielle_tiles)

16012

In [69]:
maybe_LDS = Gabrielle[(Gabrielle.match_score < 100)].sort_values("match_score")
maybe_LDS

,filename,matched_image,match_score
30123,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737
31483,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529
30331,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459
30534,Gabrielle/Shorelines/Delivery/PostGabrielle_shorelines_21022023.shp,Gabrielle/Imagery/pre-storm/Bay of Plenty/Tauranga/tauranga-winter-01m-urban-aerial-photos-2022/BD37_500_022023.jpg,60.000000
30053,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462
...,...,...,...
30024,Gabrielle/Shorelines/Waikato/PauanuiTairua/TairauPauanui_25AUG2023.shp,Gabrielle/Imagery/post_storm/Region/Waikato/Tairua/SkyFi/TairuaPauanui_25AUG2023.tif,95.454545
31313,Gabrielle/Shorelines/Northland/Ruakaka/Waipu_29JUN2021.shp,Gabrielle/Imagery/pre-storm/Northland/Waipu/Waipu_29JUNE2021.tif,96.551724
30771,Gabrielle/Shorelines/Auckland/Managawhai/Mangawhai_29JUN2021.shp,Gabrielle/Imagery/pre-storm/Northland/Mangawhai/Mangawhai_29JUNE2021.tif,97.297297
30861,Gabrielle/Shorelines/Auckland/Whangapoua/Whangapoua_09FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Whangapoua/Whangapoua_19FEB2023.tif,97.297297


In [68]:
for filename in tqdm(maybe_LDS.filename):
  print(filename)
  gdf = gpd.read_file(prefix+filename)
  bounds = gdf.total_bounds
  date = gdf.Date.unique()[0]
  DSASdate = gdf.DSASDate.unique()[0]
  match, score, index = rapidfuzz.process.extractOne(query=date, choices=Gabrielle_tiles.FLOWN.unique())
  if score != 100:
    print(f"{date} is likely a typo for {match} ({score}% match)")
  tiles_from_this_date = Gabrielle_tiles[Gabrielle_tiles.FLOWN == match]
  intersecting_tiles = tiles_from_this_date[tiles_from_this_date.intersects(box(*bounds))]
  assert len(intersecting_tiles) > 0
  resolutions = intersecting_tiles.GSDM.str.strip("m").astype(float).unique()
  print(resolutions[0])

  0%|          | 0/71 [00:00<?, ?it/s]

Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp
2023-02-18 is likely a typo for 2023-03-31 (80.0% match)


AssertionError: 

In [44]:
m = Gabrielle_tiles[Gabrielle_tiles.FLOWN.str.contains("2023", na=False)].explore()
gdf.explore(m=m)
m

In [ ]:
gpd.read_file(prefix+index_tiles.iloc[0])

,TILENAME,MAPSHEET,SCALE,TILE,geometry
0,BH41_1000_0924,BH41,1000,0924,"POLYGON ((1983520.000 5687520.000, 1983040.000 5687520.000, 1983040.000 5688240.000, 1983520.000 5688240.000, 1983520.000 5687..."
1,BH41_1000_0925,BH41,1000,0925,"POLYGON ((1984000.000 5687520.000, 1983520.000 5687520.000, 1983520.000 5688240.000, 1984000.000 5688240.000, 1984000.000 5687..."
2,BH41_1000_0926,BH41,1000,0926,"POLYGON ((1984480.000 5687520.000, 1984000.000 5687520.000, 1984000.000 5688240.000, 1984480.000 5688240.000, 1984480.000 5687..."
3,BH41_1000_0927,BH41,1000,0927,"POLYGON ((1984960.000 5687520.000, 1984480.000 5687520.000, 1984480.000 5688240.000, 1984960.000 5688240.000, 1984960.000 5687..."
4,BH41_1000_0928,BH41,1000,0928,"POLYGON ((1985440.000 5687520.000, 1984960.000 5687520.000, 1984960.000 5688240.000, 1985440.000 5688240.000, 1985440.000 5687..."
...,...,...,...,...,...
6155,BL39_1000_0104,BL39,1000,0104,"POLYGON ((1925920.000 5585280.000, 1925440.000 5585280.000, 1925440.000 5586000.000, 1925920.000 5586000.000, 1925920.000 5585..."
6156,BL39_1000_0201,BL39,1000,0201,"POLYGON ((1924480.000 5584560.000, 1924000.000 5584560.000, 1924000.000 5585280.000, 1924480.000 5585280.000, 1924480.000 5584..."
6157,BL39_1000_0202,BL39,1000,0202,"POLYGON ((1924960.000 5584560.000, 1924480.000 5584560.000, 1924480.000 5585280.000, 1924960.000 5585280.000, 1924960.000 5584..."
6158,BL39_1000_0203,BL39,1000,0203,"POLYGON ((1925440.000 5584560.000, 1924960.000 5584560.000, 1924960.000 5585280.000, 1925440.000 5585280.000, 1925440.000 5584..."


In [ ]:
BOPLINZ_footprints = gpd.read_file(prefix+"Gabrielle/")

In [ ]:
Gabrielle[Gabrielle.filename.str.contains("LINZ")].sort_values("filename")

,filename,matched_image,match_score
60,Gabrielle/Shorelines/BayofPlenty/Ohope/BOPLINZ_Ohope_Whaka_20MAR2023.shp,Gabrielle/Imagery/post_storm/Region/Hawkes Bay/Nuhaka/Nuhaka_08MAR2023.tif,73.333333
49,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737
51,Gabrielle/Shorelines/BayofPlenty/Papamoa/BOPLINZ_MtMaunganui_15MAR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Tawharanui/Tawharanui_01MAR2023.tif,69.677419
53,Gabrielle/Shorelines/BayofPlenty/Papamoa/BOPLINZ_MtMaunganui_23MAR2023.shp,Gabrielle/Imagery/post_storm/Region/Northland/Ngururu/Ngunguru_25MAR2023.tif,67.500000
52,Gabrielle/Shorelines/BayofPlenty/Papamoa/BOPLINZ_Papamoa_04APR2023.shp,Gabrielle/Imagery/pre-storm/Northland/Pataua North/Pataua_07APR2022.tif,64.285714
63,Gabrielle/Shorelines/BayofPlenty/Pukehina/BOPLINZ_Matata_24MAR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Matata/Matata_03MAR2023.tif,77.142857
62,Gabrielle/Shorelines/BayofPlenty/Pukehina/BOPLINZ_Matata_31MAR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Matata/Matata_03MAR2023.tif,83.571429
42,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462


In [ ]:
Gabrielle[Gabrielle.filename.str.contains("Gisborne") & (Gabrielle.match_score < 100)].sort_values("filename")

,filename,matched_image,match_score
226,Gabrielle/Shorelines/Gisborne/Gisborne/Gisborne_07MAR2023.shp,Gabrielle/Imagery/post_storm/Region/Gisborne/Gisborne/Gisborne_21FEB2023.tif,72.727273
225,Gabrielle/Shorelines/Gisborne/Gisborne/Gisborne_16FEB2022.shp,Gabrielle/Imagery/post_storm/Region/Gisborne/Gisborne/Gisborne_21FEB2023.tif,88.235294
222,Gabrielle/Shorelines/Gisborne/HicksBay/HicksBay_01Mar2023.shp,Gabrielle/Imagery/post_storm/Region/Gisborne/TeAraroaHicksBay/TeAraroaHicksBay_01MAR2023.tif,90.000000
221,Gabrielle/Shorelines/Gisborne/HicksBay/HicksBay_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Gisborne/HicksBay/HicksBay_08FEB2023.tif,78.787879
202,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529
203,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_22MAR2023.shp,Gabrielle/Imagery/post_storm/Region/Northland/Pataua/Pataua_25MAR2023.tif,68.750000
204,Gabrielle/Shorelines/Gisborne/TeAraroa/TeAraroa_01MAR2023.shp,Gabrielle/Imagery/post_storm/Region/Gisborne/TeAraroaHicksBay/TeAraroaHicksBay_01MAR2023.tif,80.000000
201,Gabrielle/Shorelines/Gisborne/TeAraroa/TeAraroa_19DEC2021.shp,Gabrielle/Imagery/pre-storm/Gisborne/TeAraroa/TeAraroa_08FEB2023.tif,72.727273
205,Gabrielle/Shorelines/Gisborne/TeAraroa/TeAraroa_25APR2023.shp,Gabrielle/Imagery/pre-storm/Gisborne/TeAraroa/TeAraroa_08FEB2023.tif,72.727273
220,Gabrielle/Shorelines/Gisborne/TokomaruBay/TokomaruBay_08FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Gisborne/TokomaruBay/TokomaruBay_21FEB2023.tif,92.307692


In [ ]:
df = pd.concat([df,Gabrielle])

In [48]:
df[df.filename == "MaxarImagery/HighFreq/Southland/ColacBay/Shorelines/ColacBay_19SEP2007.shp"]

,filename,matched_image,match_score
42501,MaxarImagery/HighFreq/Southland/ColacBay/Shorelines/ColacBay_19SEP2007.shp,MaxarImagery/HighFreq/Southland/ColacBay/Imagery/Stack/ColacBay_04SEP2018.tif,78.787879


In [49]:
df[df.filename.str.contains("Whatipu_07APR2010")]

,filename,matched_image,match_score
50453,MaxarImagery/HighFreq/Auckland/Whatipu/Shorelines/Whatipu_07APR2010.shp,Retrolens/Auckland/Whatipu/Stack/Whatipu_14APR1940_mosaic.tif,77.419355


In [12]:
df = df[df.match_score >= 100].sort_values(by="match_score")
df

,filename,matched_image,match_score
32067,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_31AUG2005.shp,MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif,100.0
154456,Retrolens/Otago/Waikouaiti/Shorelines/Waikouaiti_01MAR1969.shp,Retrolens/Otago/Waikouaiti/Stack/Waikouaiti_01MAR1969_mosaic.jp2,100.0
154450,Retrolens/Otago/Waikouaiti/Shorelines/Waikouaiti_18FEB1975.shp,Retrolens/Otago/Waikouaiti/Stack/Waikouaiti_18FEB1975_mosaic.jp2,100.0
154439,Retrolens/Otago/Waikouaiti/Shorelines/Waikouaiti_24FEB1958.shp,Retrolens/Otago/Waikouaiti/Stack/Waikouaiti_24FEB1958_mosaic.jp2,100.0
154233,Retrolens/Otago/CoalPoint_SmithsBeach/Shorelines/CoalPoint_SmithsBeach_16APR1982.shp,Retrolens/Otago/CoalPoint_SmithsBeach/Stack/CoalPoint_SmithsBeach_16APR1982_mosaic.jp2,100.0
...,...,...,...
78626,Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_26SEP1972.shp,Retrolens/HawkesBay/Awatoto/Stack/Awatoto_26SEP1972_mosaic.jp2,100.0
78613,Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_26NOV1988.shp,Retrolens/HawkesBay/Awatoto/Stack/Awatoto_26NOV1988_mosaic.jp2,100.0
78612,Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_29SEP1977.shp,Retrolens/HawkesBay/Awatoto/Stack/Awatoto_29SEP1977_mosaic.jp2,100.0
79288,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_25MAR1965.shp,Retrolens/HawkesBay/Porangahau/Stack/Porangahau_25MAR1965_mosaic.jp2,100.0


## Investigate metadata about the matched images

In [50]:
gpd.read_file("Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_08MAR1956.shp")

,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,geometry
0,0,Otago,Ryans_Pipikaretu_Penguin_TeRauoneBeach,1956-03-08,08/03/1956,TK,1250,EOV,RL,2.0,1.0,18100,2.09,0.636996,2.303641,"LINESTRING (1423319.183 4926867.847, 1423320.091 4926862.866, 1423319.579 4926860.306, 1423320.373 4926857.925, 1423320.373 49..."
1,0,Otago,Ryans_Pipikaretu_Penguin_TeRauoneBeach,1956-03-08,08/03/1956,TK,1500,EOV,RL,2.0,1.0,18100,2.09,0.636996,2.303641,"LINESTRING (1424401.205 4927041.507, 1424390.490 4927037.538, 1424384.537 4927033.966, 1424382.949 4927031.585, 1424380.965 49..."
2,0,Otago,Ryans_Pipikaretu_Penguin_TeRauoneBeach,1956-03-08,08/03/1956,TK,1500,EOV,RL,NaN,NaN,18100,2.09,0.636996,NaN,None
3,0,Otago,Ryans_Pipikaretu_Penguin_TeRauoneBeach,1956-03-08,08/03/1956,TK,1500,EOV,RL,3.0,1.0,18100,2.09,0.636996,2.390557,"LINESTRING (1424467.817 4925745.867, 1424466.879 4925745.206, 1424463.848 4925745.206, 1424460.541 4925740.576, 1424456.572 49..."
4,0,Otago,Ryans_Pipikaretu_Penguin_TeRauoneBeach,1956-03-08,08/03/1956,TK,1250,EOV,RL,1.0,1.0,18100,2.09,0.636996,2.226828,"LINESTRING (1424679.042 4924650.162, 1424677.388 4924647.185, 1424676.396 4924644.539, 1424674.742 4924642.886, 1424672.758 49..."
5,0,Otago,Ryans_Pipikaretu_Penguin_TeRauoneBeach,1956-03-08,08/03/1956,TK,1250,EOV,RL,1.0,1.0,18100,2.09,0.636996,2.226828,"LINESTRING (1424618.518 4924476.859, 1424618.187 4924475.536, 1424617.195 4924473.221, 1424617.526 4924471.567, 1424617.526 49..."
6,0,Otago,Ryans_Pipikaretu_Penguin_TeRauoneBeach,1956-03-08,08/03/1956,TK,1000,Base of hillslope,RL,1.0,1.0,18100,2.09,0.636996,2.226828,"LINESTRING (1424289.924 4926149.681, 1424288.623 4926147.860, 1424287.193 4926146.169, 1424285.632 4926144.869, 1424284.591 49..."


In [14]:
def get_meta(tup):
    i, row = tup
    image = rio.open(prefix + row.matched_image)
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.explode(index_parts=False))
    except: 
        print(f"Can't read{row['filename']}")
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, df.iterrows(), total=len(df)))
    meta.to_csv(metafile, index=False)
meta

,filename,matched_image,match_score,n_lines,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,compress,interleave,GCPs,res,CPS,photometric
0,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_31AUG2005.shp,MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif,100.0,2,GTiff,uint8,NaN,3975,12039,3,NaN,"| 0.60, 0.00, 2022707.13|\n| 0.00,-0.60, 5670278.45|\n| 0.00, 0.00, 1.00|",128.0,128,True,lzw,pixel,26,"(0.6, 0.5999999999999536)",True,NaN
1,Retrolens/Otago/Waikouaiti/Shorelines/Waikouaiti_01MAR1969.shp,Retrolens/Otago/Waikouaiti/Stack/Waikouaiti_01MAR1969_mosaic.jp2,100.0,3,JP2OpenJPEG,uint16,256.0,2694,7132,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""G...","| 0.66, 0.00, 1416855.81|\n| 0.00,-0.66, 4946373.69|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,pixel,0,"(0.6609072570840094, 0.6609072570839978)",True,NaN
2,Retrolens/Otago/Waikouaiti/Shorelines/Waikouaiti_18FEB1975.shp,Retrolens/Otago/Waikouaiti/Stack/Waikouaiti_18FEB1975_mosaic.jp2,100.0,2,JP2OpenJPEG,uint16,256.0,4192,4051,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""G...","| 0.72, 0.00, 1417191.69|\n| 0.00,-0.72, 4947057.65|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,pixel,0,"(0.7215191398635404, 0.7215191398636224)",True,NaN
3,Retrolens/Otago/Waikouaiti/Shorelines/Waikouaiti_24FEB1958.shp,Retrolens/Otago/Waikouaiti/Stack/Waikouaiti_24FEB1958_mosaic.jp2,100.0,4,JP2OpenJPEG,uint16,256.0,7247,11646,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""G...","| 0.47, 0.00, 1416749.83|\n| 0.00,-0.47, 4946987.89|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,pixel,0,"(0.4674722337135327, 0.46747223371356755)",True,NaN
4,Retrolens/Otago/CoalPoint_SmithsBeach/Shorelines/CoalPoint_SmithsBeach_16APR1982.shp,Retrolens/Otago/CoalPoint_SmithsBeach/Stack/CoalPoint_SmithsBeach_16APR1982_mosaic.jp2,100.0,9,JP2OpenJPEG,uint16,256.0,6946,5747,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""G...","| 0.97, 0.00, 1360458.60|\n| 0.00,-0.97, 4871520.91|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,pixel,0,"(0.9661514817232834, 0.9661514817232894)",True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_26SEP1972.shp,Retrolens/HawkesBay/Awatoto/Stack/Awatoto_26SEP1972_mosaic.jp2,100.0,8,JP2OpenJPEG,uint16,256.0,6342,22895,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""G...","| 0.23, 0.00, 1936470.34|\n| 0.00,-0.23, 5617300.14|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,pixel,0,"(0.23214579740359595, 0.2321457974035899)",True,NaN
1605,Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_26NOV1988.shp,Retrolens/HawkesBay/Awatoto/Stack/Awatoto_26NOV1988_mosaic.jp2,100.0,8,JP2OpenJPEG,uint16,256.0,4408,11592,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""G...","| 0.74, 0.00, 1935971.56|\n| 0.00,-0.74, 5617932.02|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,pixel,0,"(0.7384080175577009, 0.7384080175576948)",True,NaN
1606,Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_29SEP1977.shp,Retrolens/HawkesBay/Awatoto/Stack/Awatoto_29SEP1977_mosaic.jp2,100.0,9,JP2OpenJPEG,uint16,256.0,4994,11676,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""G...","| 0.71, 0.00, 1935914.35|\n| 0.00,-0.71, 5617032.39|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,pixel,0,"(0.7072780690590255, 0.7072780690590317)",True,NaN
1607,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_25MAR19

In [15]:
gpd.read_file("Retrolens/Northland/Owhata/Shorelines/Owhata_03OCT1981.shp")

,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,geometry


In [16]:
empty = meta[meta.n_lines == 0]
empty.shape

(78, 21)

In [18]:
def get_mtime(filename):
    return pd.to_datetime(os.path.getmtime(prefix+filename), unit="s", origin="unix", utc=True).tz_convert("Pacific/Auckland")
empty["mtime"] = empty.filename.apply(get_mtime)
empty["size_bytes"] = (prefix + empty.filename).apply(os.path.getsize)
#pd.set_option("display.max_rows",None)
empty[["filename", "n_lines", "mtime", "size_bytes"]].sort_values("mtime")

/tmp/ipykernel_1800137/225968677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty["mtime"] = empty.filename.apply(get_mtime)
/tmp/ipykernel_1800137/225968677.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty["size_bytes"] = (prefix + empty.filename).apply(os.path.getsize)


,filename,n_lines,mtime,size_bytes
1505,MaxarImagery/HighFreq/WestCoast/Ohinemaka/Shorelines/Ohinemaka_14MAR2015.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1156,MaxarImagery/HighFreq/Southland/Riverton/Shorelines/Riverton_27Dec2015.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1246,Retrolens/Southland/Riverton/Shorelines/Riverton_11Feb1978.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1247,Retrolens/Southland/Riverton/Shorelines/Riverton_10Dec1958.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1248,Retrolens/Southland/Riverton/Shorelines/Riverton_24Feb1968.shp,0,2021-06-16 12:17:29.273264896+12:00,100
...,...,...,...,...
842,MaxarImagery/HighFreq/Canterbury/Washdyke/Shorelines/Washdyke_16AUG2017.shp,0,2022-01-16 15:09:57.307988992+13:00,100
841,MaxarImagery/HighFreq/Canterbury/Washdyke/Shorelines/Washdyke_24FEB2019.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1396,Retrolens/Taranaki/New Plymouth District Council/Waitara/Shorelines/Waitara_16SEP1958.shp,0,2022-05-06 20:21:39.995812864+12:00,100
1386,Retrolens/Taranaki/South Taranaki/CapeEgmont/Shorelines/CapeEgmont_06SEP1965.shp,0,2022-05-16 22:49:02.900338944+12:00,100


In [28]:
with pd.option_context("display.max_rows", 70):
  display(empty[["filename", "n_lines", "mtime", "size_bytes"]][empty.mtime > "2021-11-23"].sort_values("mtime"))

,filename,n_lines,mtime,size_bytes
105,Retrolens/Northland/Owhata/Shorelines/Owhata_03OCT1981.shp,0,2022-01-16 15:09:57.307988992+13:00,100
973,MaxarImagery/HighFreq/Otago/Oamaru/Shorelines/Oamaru_06JAN2009.shp,0,2022-01-16 15:09:57.307988992+13:00,100
975,MaxarImagery/HighFreq/Otago/Oamaru/Shorelines/Oamaru_06MAY2015.shp,0,2022-01-16 15:09:57.307988992+13:00,100
976,MaxarImagery/HighFreq/Otago/Oamaru/Shorelines/Oamaru_04APR2018.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1040,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_04MAR2018.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1043,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_28AUG2014.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1044,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_07JUNE2016.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1045,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_11APR2020.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1051,MaxarImagery/HighFreq/BayOfPlenty/OhopeBeach/Shorelines/OhopeBeach_16MAY2014.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1071,MaxarImagery/HighFreq/HawkesBay/TableCape/Shorelines/TableCape_04DEC2002.shp,0,2022-01-16 15:09:57.307988992+13:00,100


In [ ]:
meta[meta.filename.str.startswith("Gabrielle")].CPS.value_counts()

In [ ]:
meta.crs.value_counts(dropna=False)

In [ ]:
meta.GCPs.value_counts()

In [ ]:
meta.columns

In [ ]:
meta.driver.value_counts()

In [ ]:
meta["count"].value_counts()

In [ ]:
meta.dtype.value_counts()

In [ ]:
meta.nodata.value_counts()

## Make mosaics for LINZ images

In [ ]:
maybe_LDS = df[(df.match_score < 100) & df.filename.str.startswith("Retrolens")].copy()
maybe_LDS

In [ ]:
if not os.path.isfile("maybe_LDS.csv"):
    maybe_LDS.filename.to_csv("maybe_LDS.csv", index=False)

## Match shapefiles with the corresponding index tiles shapefile
- First get the bounds of every tile
- Tiles that spatially match the bounds of a drawn EOV shapefile will be used to create the corresponding mosaic

In [ ]:
if os.path.isfile("tilelist.parquet"):
    tilelist = gpd.read_parquet("tilelist.parquet")
else:
    tilelist = pd.DataFrame({"filename": glob("DigitalJPGs/**/*.jpg", recursive=True)})
    tilelist["region"] = tilelist.filename.str.split("/").str[1]
    tilelist["tilename"] = tilelist.filename.str.split("/").str[-1].str.replace(".jpg", "")
    def get_bounds(f):
        return rio.open(f).bounds
    tilelist["bounds"] = thread_map(get_bounds, tilelist.filename)
    tilelist.bounds = tilelist.bounds.progress_apply(lambda b: box(*b))
    tilelist = gpd.GeoDataFrame(tilelist, geometry="bounds")
    tilelist.to_parquet("tilelist.parquet")

In [ ]:
# This cell might useful for finding matches, based on geospatial correlation
for filename in tqdm(maybe_LDS.filename):
    break
    df = gpd.read_file(filename)
    if len(df) == 0:
        continue
    bounds = df.total_bounds
    intersecting_tiles = tilelist[tilelist.intersects(box(*bounds))]
    print(f"{filename} matches:\n\t{len(intersecting_tiles)} tiles from:\n\t\t{intersecting_tiles.filename}")

In [ ]:
LDS = pd.read_csv("maybe_LDS.csv").dropna()
LDS

For each file, create a mosaic from the corresponding tiles

In [ ]:
def get_match(filename):
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=df.filename, processor=fuzz_preprocess)
    return match, score
LDS["matched_filename"], LDS["match_score"] = zip(*LDS.filename.apply(get_match))
LDS[LDS.filename != LDS.matched_filename][["filename", "matched_filename", "match_score"]]

In [ ]:
LDS.filename = LDS.matched_filename

In [ ]:
done = LDS.matched_filename.apply(lambda f: os.path.isfile(prefix + f.replace(".shp", ".tif")))
done.value_counts(dropna=False)

In [ ]:
done = LDS.matched_filename.apply(lambda f: os.path.isfile(prefix + f.replace(".shp", ".tif")))
display(done.value_counts(dropna=False))

for i, row in tqdm(LDS[~done].iterrows(), total=len(LDS[~done])):
    filename = row.matched_filename
    mosaic_filename = filename.replace(".shp", ".tif")
    shapefile = gpd.read_file(filename)
    bounds = shapefile.total_bounds
    intersecting_tiles = tilelist[tilelist.intersects(box(*bounds)) & tilelist.filename.str.startswith(row.matched_tile_root)]
    tiles = list(intersecting_tiles.filename)
    print(len(tiles))
    Z, transform = merge(tiles)
    with rasterio.open(
        mosaic_filename,
        'w',
        driver='GTiff',
        height=Z.shape[1],
        width=Z.shape[2],
        count=Z.shape[0],
        dtype=Z.dtype,
        crs=shapefile.crs,
        transform=transform,
        compress='lzw',
        BIGTIFF = "IF_SAFER"
    ) as dst:
        dst.write(Z)

In [ ]:
LDS["matched_image"] = LDS.filename.str.replace(".shp", ".tif")
LDS.to_csv("LDS_matches.csv", index=False)

In [ ]:
metafile = "LDS_meta.csv"
if os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, LDS.iterrows(), total=len(LDS)))
    meta.to_csv(metafile, index=False)
meta

### Algorithm for converting polyline shapefile to polygon annotations, labelled as sea or land

In [ ]:
coastline = gpd.read_file("lds-nz-coastlines-and-islands-polygons-topo-150k-FGDB.zip!nz-coastlines-and-islands-polygons-topo-150k.gdb")

In [ ]:
# Get a random (known-good) annotation
sample = LDS.sample(1)
display(sample)
image_filename = sample.matched_image.iloc[0]
image = rio.open(image_filename)
sample_gdf = gpd.read_file(sample.filename.iloc[0])
sample_gdf

In [ ]:
def line_to_split_bbox(geo):
    bounding_box = geo.envelope
    split_bbox = shapely.ops.split(bounding_box, geo)
    return split_bbox

split_bboxes = sample_gdf.geometry.apply(line_to_split_bbox).explode(index_parts=True).reset_index()
#split_bboxes.geometry = split_bboxes.geometry.buffer(0)
split_bboxes["area"] = split_bboxes.area
split_bboxes = split_bboxes[split_bboxes.area > 1e5]
split_bboxes

In [ ]:
relevant_coastline = coastline.clip(split_bboxes.total_bounds)
split_bboxes["area_inland"] = split_bboxes.clip(relevant_coastline).area
split_bboxes["fraction_inland"] = split_bboxes.area_inland / split_bboxes.area
split_bboxes["class"] = split_bboxes.fraction_inland.apply(lambda f: "land" if f > .5 else "sea")
split_bboxes

In [ ]:
# Plot the results, and check it all looks ok
fig, ax = plt.subplots()
ax = rasterio.plot.show(image, ax=ax)

cmap = matplotlib.colors.ListedColormap(['green', 'blue'])
split_bboxes.plot(ax=ax, alpha=.5, column='class', cmap=cmap, categorical=True, legend=True, edgecolor='black')
split_bboxes.apply(lambda x: ax.annotate(text=round(x.fraction_inland, 2), xy=x.geometry.centroid.coords[0], ha='center'), axis=1)

#relevant_coastline.plot(ax=ax, alpha=.5, edgecolor="cyan")

b = split_bboxes.total_bounds
xlim = ([b[0], b[2]])
ylim = ([b[1], b[3]])
ax.set_xlim(xlim)
ax.set_ylim(ylim)